In [20]:
from joblib import dump, load
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk as nlp
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression


In [22]:
new_df = pd.read_csv('data.csv')

In [23]:
new_df.head()

,article_id,source_id,source_name,author,title,description,url,url_to_image,published_at,content,category,full_content
0,89541,NaN,International Business Times,Paavan MATHEMA,UN Chief Urges World To 'Stop The Madness' Of ...,UN Secretary-General Antonio Guterres urged th...,https://www.ibtimes.com/un-chief-urges-world-s...,https://d.ibtimes.com/en/full/4496078/nepals-g...,2023-10-30 10:12:35.000000,UN Secretary-General Antonio Guterres urged th...,Nepal,UN Secretary-General Antonio Guterres urged th...
1,89542,NaN,Prtimes.jp,NaN,RANDEBOOよりワンランク上の大人っぽさが漂うニットとベストが新登場。,[株式会社Ainer]\nRANDEBOO（ランデブー）では2023年7月18日(火)より公...,https://prtimes.jp/main/html/rd/p/000000147.00...,https://prtimes.jp/i/32220/147/ogp/d32220-147-...,2023-10-06 04:40:02.000000,"RANDEBOO2023718()WEB2023 Autumn Winter \n""Nepa...",Nepal,NaN
2,89543,NaN,VOA News,webdesk@voanews.com (Agence France-Presse),UN Chief Urges World to 'Stop the Madness' of ...,UN Secretary-General Antonio Guterres urged th...,https://www.voanews.com/a/un-chief-urges-world...,https://gdb.voanews.com/01000000-0a00-0242-60f...,2023-10-30 10:53:30.000000,"Kathmandu, Nepal UN Secretary-General Antonio...",Nepal,NaN
3,89545,NaN,The Indian Express,Editorial,Sikkim warning: Hydroelectricity push must be ...,Ecologists caution against the adverse effects...,https://indianexpress.com/article/opinion/edit...,https://images.indianexpress.com/2023/10/edit-...,2023-10-06 01:20:24.000000,At least 14 persons lost their lives and more ...,Nepal,At least 14 persons lost their lives and more ...
4,89547,NaN,The Times of Israel,Jacob Magid,"200 foreigners, dual nationals cut down in Ham...","France lost 35 citizens, Thailand 33, US 31, U...",https://www.timesofisrael.com/200-foreigners-d...,https://static.timesofisrael.com/www/uploads/2...,2023-10-27 01:08:34.000000,"Scores of foreign citizens were killed, taken ...",Nepal,NaN


In [ ]:
# Load the trained Logistic Regression model
lr = load('logistic_regression_model.joblib')

# Load the TF-IDF vectorizer
vectorization = load('tfidf_vectorizer.joblib')

In [25]:
# Define the preprocessing function
def preprocess_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[()]', '', text)
    text = re.sub('\W', ' ', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', ' ', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [26]:
# Apply preprocessing to the "content" column and create a new column called "processed_content"
new_df['processed_content'] = new_df['content'].apply(preprocess_text)

# Vectorize the preprocessed text data
new_x = new_df['processed_content']
new_xv = vectorization.transform(new_x)

# Make predictions using the trained Logistic Regression model
predictions = lr.predict(new_xv)

# Add predictions to the dataframe
new_df['predicted_label'] = predictions

# Output the dataframe with predictions
print(new_df[['content', 'predicted_label']])

                                                  content  predicted_label
0       UN Secretary-General Antonio Guterres urged th...                0
1       RANDEBOO2023718()WEB2023 Autumn Winter \n"Nepa...                0
2       Kathmandu, Nepal  UN Secretary-General Antonio...                0
3       At least 14 persons lost their lives and more ...                0
4       Scores of foreign citizens were killed, taken ...                1
...                                                   ...              ...
105370  Karnataka Deputy Chief Minister D K Shivakumar...                1
105371  FC Barcelona have guaranteed at least $767.6 m...                0
105372  The photo from David and Sarah Lubarsky's wedd...                0
105373  Kerbers Farm: Bringing Farm To Table To Manhat...                0
105374  Cofounder at UpperKey. Passionate about proper...                0

[105375 rows x 2 columns]


In [29]:
new_df['predicted_label'].value_counts()

0    86109
1    19266
Name: predicted_label, dtype: int64